In [17]:
"""Test script that runs all the vehicles in both Excel and Python FASTSim for both UDDS and HWFET cycles."""

import xlwings as xw
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import importlib
import xlwings as xw
from math import isclose
import importlib
import pickle

# local modules
from fastsim.tests import test_vs_excel
importlib.reload(test_vs_excel)
from fastsim import cycle, vehicle, simdrive, simdrivelabel
importlib.reload(simdrivelabel)

<module 'fastsim.simdrivelabel' from 'c:\\users\\cbaker2\\documents\\projects\\fastsim\\github\\fastsim\\fastsim\\simdrivelabel.py'>

In [18]:
# Ford C-Max PHEV
veh = vehicle.Vehicle(14).get_numba_veh()
cyc = cycle.Cycle('udds').get_numba_cyc()
sd = simdrive.SimDriveJit(cyc, veh)
sd.sim_drive(veh.maxSoc)
label_fe, sds = simdrivelabel.get_label_fe(veh, full_detail=True)
udds = label_fe['phev_calcs']['udds']
hwy = label_fe['phev_calcs']['hwy']

In [19]:
res_python = test_vs_excel.run_python(verbose=False, use_jit=True)
# res_python = {'2016 FORD C-MAX (PHEV)':res_python['2016 FORD C-MAX (PHEV)']}

Instantiating classes.


Elapsed time:  0.92 s


In [20]:
res_excel = test_vs_excel.run_excel('res_excel.p')

In [21]:
res_comps = test_vs_excel.compare(res_python, res_excel)



2016 TOYOTA Corolla 4cyl 2WD
*********************
All values within error tolerance of 0.001


2017 Prius Prime
*********************
adjUddsMpgge error = -0.909%
adjCombMpgge error = -0.493%


2016 Nissan Leaf 30 kWh
*********************
All values within error tolerance of 0.001


2016 Hyundai Tucson Fuel Cell
*********************
All values within error tolerance of 0.001


2017 CHEVROLET Bolt
*********************
All values within error tolerance of 0.001


2016 CHEVROLET Volt
*********************
All values within error tolerance of 0.001


2016 CHEVROLET Malibu 4cyl 2WD
*********************
All values within error tolerance of 0.001


2016 TOYOTA Camry 4cyl 2WD
*********************
All values within error tolerance of 0.001


2016 MITSUBISHI i-MiEV
*********************
All values within error tolerance of 0.001


2016 TESLA Model S60 2WD
*********************
All values within error tolerance of 0.001


2016 HYUNDAI Elantra 4cyl 2WD
*********************
All values with

In [6]:
vehstr = "2016 AUDI A3 4cyl 2WD"
for key in res_excel[vehstr].keys():
    print(key + ' : {', end='')
    print('excel : {:.4g}, '.format(res_excel[vehstr][key]), end='')
    print('python:{:.4g}'.format(res_python[vehstr][key]) + '}')

labUddsMpgge : {excel : 39.98, python:39.98}
labHwyMpgge : {excel : 49.96, python:49.96}
labCombMpgge : {excel : 43.93, python:43.93}
labUddsKwhPerMile : {excel : 0, python:0}
labHwyKwhPerMile : {excel : 0, python:0}
labCombKwhPerMile : {excel : 0, python:0}
adjUddsMpgge : {excel : 30.5, python:30.5}
adjHwyMpgge : {excel : 35.3, python:35.3}
adjCombMpgge : {excel : 32.49, python:32.49}
adjUddsKwhPerMile : {excel : 0, python:0}
adjHwyKwhPerMile : {excel : 0, python:0}
adjCombKwhPerMile : {excel : 0, python:0}


In [7]:
vehstr = '2016 FORD C-MAX (PHEV)'
for key in res_excel[vehstr].keys():
    print(key + ' : {', end='')
    try:
        print('excel : {:.4g}, '.format(res_excel[vehstr][key]), end='')
        print('python:{:.4g}'.format(res_python[vehstr][key]) + '}')
    except:
        print('Not a scalar}')

labUddsMpgge : {excel : 125.7, python:125.7}
labHwyMpgge : {excel : 97.49, python:97.49}
labCombMpgge : {excel : 111.2, python:111.2}
labUddsKwhPerMile : {excel : 0.1916, python:0.1916}
labHwyKwhPerMile : {excel : 0.1972, python:0.1972}
labCombKwhPerMile : {excel : 0.1941, python:0.1941}
adjUddsMpgge : {excel : 71.71, python:666}
adjHwyMpgge : {excel : 58.18, python:666}
adjCombMpgge : {excel : 64.92, python:666}
adjUddsKwhPerMile : {excel : 0.3187, python:666}
adjHwyKwhPerMile : {excel : 0.3121, python:666}
adjCombKwhPerMile : {excel : 0.3158, python:666}
